# For NSE

- [x] Make a list of .DAT files for extraction
   ... it would be like `C_VAR1_22042024_1.DAT`. There are 6 files in a day.

- [x] Download latest .DAT file from data-link `https://nsearchives.nseindia.com/archives/nsccl/var/C_VAR1_22042024_1.DAT`
- [ ] Extract the margin for the symbols. This should include NIFTY.
- [ ] Convert symbols to ib format
- [ ] Store the margin percentage for each symbol in `.pkl`


## get all report dump

In [ ]:
from utils import get_nse_payload
import io
import pandas as pd
from typing import Union
import requests

In [ ]:
def get_parsed_response_df(response: requests.models.Response) -> pd.DataFrame:
    """Parses responses from `get_nse_payload` and returns df"""

    try:
        out = pd.read_csv(io.StringIO(response.text))
    except pd.errors.ParserError:
        out = pd.DataFrame([])

    return out

In [ ]:
def get_raw_nse_margin_df() -> pd.DataFrame:
    """Returns raw df of margins from nse website"""

    base_url = 'https://nsearchives.nseindia.com/archives/nsccl/var/C_VAR1_'
    date_url = '22042024' # Loop through formatted_dates
    hyphen = '_'
    serial = str(1) # loop through `reversed(range(1, 7))`
    end_url = '.DAT'

    date_range = pd.date_range(end=pd.Timestamp.today()+pd.Timedelta(days=3), periods=7)
    formatted_dates = [date.strftime("%d%m%Y") for date in reversed(date_range)]

    serial_range = [i for i in reversed(range(1, 7))]

    for date in formatted_dates:
        date_url = date
        for i in serial_range:
            serial = str(i)
            url = base_url+date_url+hyphen+serial+end_url
            
            response = get_nse_payload(url)
            df = get_parsed_response_df(response)
            if not df.empty:
                # print(url)
                # print(f"got df! {df.head(1)}")
                return df


In [ ]:
raw_df = get_raw_nse_margin_df()

In [ ]:
df = raw_df.reset_index().iloc[:, [1, 2, 6, 8]]
df.columns = ['symbol', 'type', 'pct1', 'pct2']


In [ ]:
from utils import get_nse_native_fno_list

fno_list = get_nse_native_fno_list()
m = df.symbol.isin(fno_list) & (df.type == 'EQ')
df = df[m].reset_index(drop=True)

## Computing margins from SAMCO

In [ ]:
# variable
underlying = "AARTIIND"
expiry = "25APR"
strike_price = 720

# calculated
qty = 1000
quantity = qty


# fixed:
exchange = "NFO"
product = "OPT"
option = "PE"
buy_sell = "sell"

In [ ]:
import requests
import json


In [ ]:
data = {
    "underlying": underlying,
    "expiry": expiry,
    "strike_price": strike_price,
    "qty": qty,
    "quantity": quantity,
    "exchange": exchange,
    "product": product,
    "option": option,
    "buy_sell": buy_sell
}

In [ ]:
from utils import get_lots_from_nse

## !!! Complete this !!!

In [1]:
from utils import get_nearest_weekly_expiry_nse, get_nearest_monthly_expiry_nse

expiry = get_nearest_weekly_expiry_nse()

In [2]:
expiry

Date(2024, 4, 25)

In [ ]:
def get_margin_from_samco(underlying: str, *arg, **kwargs):

    """
    Gets margin from samco.in
    
    """

    url = "https://www.samco.in/span/get_span_value"

    headers = {
        'User-Agent': 'python-requests/2.31.0', 
        'Accept-Encoding': 'gzip, deflate', 
        'Accept': '*/*', 
        'Connection': 'keep-alive', 
        "Referer": "https://www.samco.in/calculators/span-margin-calculator"}

    expiry =  kwargs.get('expiry', False)

    if not expiry:

        if 'NIFTY' in underlying:
            expiry = get_nearest_weekly_expiry_nse()
        else:
            expiry = get_nearest_monthly_expiry_nse ()


    strike_price = kwargs.get('strike_price', False)
    qty = kwargs.get('qty', False)
    buy_sell = kwargs.get('buy_sell', 'sell')


    
    
    if not data['qty']:
        qty = get_lots_from_nse().get(underlying, 0)

    quantity = qty

    exchange = "NFO"
    product = "OPT"
    option = "PE"
    buy_sell = "sell"


In [ ]:
with requests.session() as s:
    s.get(headers['Referer'])
    response = s.post(url, data=data, headers=headers)

In [ ]:
m_dict = json.loads(response.text)
  

In [ ]:
if m_dict["success"]:
    mgnCompute = m_dict['margin']['totalrequirement']
else:
    mgnCompute = None